In [4]:
import argparse

import gymnasium as gym
import numpy as np
import torch

from mani_skill.envs.sapien_env import BaseEnv
from mani_skill.utils.geometry import rotation_conversions as rot_utils
from mani_skill.utils.structs.pose import Pose
from mani_skill.utils.wrappers import RecordEpisode
from mani_skill.utils.motion_helper import generate_trajectory, compute_delta_pose, visualize_trajectory


In [5]:
env_id = "TestBench-v1"
obs_mode = "none"
sim_backend = "auto"
reward_mode = None
num_envs = 1
control_mode = "pd_ee_delta_pose"
render_mode = "human"
shader = "default"
record_dir = None 
pause = True
quiet = False
seed = 1

In [6]:
np.set_printoptions(suppress=True, precision=3)
verbose = not quiet
if seed is not None:
    np.random.seed(seed)
parallel_in_single_scene = render_mode == "human"
if render_mode == "human" and obs_mode in ["sensor_data", "rgb", "rgbd", "depth", "point_cloud"]:
    print("Disabling parallel single scene/GUI render as observation mode is a visual one. Change observation mode to state or state_dict to see a parallel env render")
    parallel_in_single_scene = False
if render_mode == "human" and num_envs == 1:
    parallel_in_single_scene = False
env: BaseEnv = gym.make(
    env_id,
    obs_mode=obs_mode,
    reward_mode=reward_mode,
    control_mode=control_mode,
    render_mode=render_mode,
    shader_dir=shader,
    num_envs=num_envs,
    sim_backend=sim_backend,
    parallel_in_single_scene=parallel_in_single_scene,
)
record_dir = record_dir
if record_dir:
    record_dir = record_dir.format(env_id=env_id)
    env = RecordEpisode(env, record_dir, info_on_video=False, save_trajectory=False, max_steps_per_video=env._max_episode_steps)

if verbose:
    print("Observation space", env.observation_space)
    print("Action space", env.action_space)
    print("Control mode", env.unwrapped.control_mode)
    print("Reward mode", env.unwrapped.reward_mode)
    for cname in env.unwrapped.agent.controller.controllers.keys():
        controller_cfg = env.unwrapped.agent.controller.controllers[cname].config
        print("Controller[", cname, "]")
        for cfg_attr in dir(controller_cfg):
            if not cfg_attr.startswith("_"):
                print("\t>>>", cfg_attr, getattr(controller_cfg, cfg_attr))

RuntimeError: input vector size does not match articulation DOFs

In [12]:
def get_ee_pose(controller, at_base = False):
    if at_base:
        pose = controller.ee_pose_at_base
    else:
        pose = controller.ee_pose
    
    pos, quat = pose.p, pose.q
    angle = rot_utils.quaternion_to_axis_angle(quat) / np.pi * 180
    
    return {'pos': pos, 'quat': quat, 'angle': angle}

In [13]:
obs, _ = env.reset(seed=seed)
env.action_space.seed(seed)
if render_mode is not None:
    viewer = env.render()
    viewer.paused = pause
    env.render()
    
print("Initial pose", get_ee_pose(env.agent.controller.controllers["arm"]))
print("Initial pose at base", get_ee_pose(env.agent.controller.controllers["arm"], at_base=True))

In [6]:
action = torch.tensor([0.5, 0.5, -1.0, 0.0, 0.0, 0.0, 0.5])
env.step(action)

ACTION tensor([[ 0.0500,  0.0500, -0.1000, -0.0000, -0.0000, -0.0000]])


({},
 None,
 tensor([False]),
 tensor([False]),
 {'elapsed_steps': tensor([1], dtype=torch.int32)})

In [8]:
rot_utils.quaternion_to_axis_angle(env.agent.controller.controllers["arm"]._target_pose.q)

/home/gaofeng/miniconda3/envs/arm/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.agent to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.agent` for environment variables or `env.get_wrapper_attr('agent')` that will search the reminding wrappers.
  logger.warn(


tensor([[2.2214e+00, 2.2214e+00, 2.9024e-06]])

In [25]:
for i in range(1000):
    print("-----")
    print("Before", get_ee_pose(env.agent.controller.controllers['arm']))
    env.scene.step()
    print("After", get_ee_pose(env.agent.controller.controllers['arm']))
    env.render()

-----
Before {'pos': tensor([[0.0096, 0.2097, 0.4097]]), 'quat': tensor([[-0.0353, -0.7061, -0.7063,  0.0353]]), 'angle': tensor([[-130.0396, -130.0743,    6.4973]])}
After {'pos': tensor([[0.0096, 0.2097, 0.4097]]), 'quat': tensor([[-0.0353, -0.7061, -0.7063,  0.0353]]), 'angle': tensor([[-130.0395, -130.0742,    6.4978]])}
-----
Before {'pos': tensor([[0.0096, 0.2097, 0.4097]]), 'quat': tensor([[-0.0353, -0.7061, -0.7063,  0.0353]]), 'angle': tensor([[-130.0395, -130.0742,    6.4978]])}
After {'pos': tensor([[0.0096, 0.2097, 0.4097]]), 'quat': tensor([[-0.0353, -0.7061, -0.7063,  0.0353]]), 'angle': tensor([[-130.0395, -130.0741,    6.4981]])}
-----
Before {'pos': tensor([[0.0096, 0.2097, 0.4097]]), 'quat': tensor([[-0.0353, -0.7061, -0.7063,  0.0353]]), 'angle': tensor([[-130.0395, -130.0741,    6.4981]])}
After {'pos': tensor([[0.0096, 0.2097, 0.4097]]), 'quat': tensor([[-0.0353, -0.7061, -0.7063,  0.0353]]), 'angle': tensor([[-130.0394, -130.0741,    6.4982]])}
-----
Before {'pos'

KeyboardInterrupt: 